In [1]:
import pandas as pd
import numpy as np
import config
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)


3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=10
year=21
sql = f"""
select ps.id, ps.nom, ps.prenom, ar.dept_id, an.id adresse_norm_id, an.iris::float, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
where sp.specialite_id=10
and tds.date_source_id >= 2100 and  tds.date_source_id < 2200
and ar.dept_id=6
group by ps.id, c.id, ar.id, an.id
"""
print(sql)
gene = pd.read_sql(sql, config.connection_string)
# 1286
# 50488 gene en 2023
# 47058 en 2024

# gene = pd.read_stata("Etude 06/Gene06.dta")
gene.style.set_table_attributes('style="font-size: 10px;height:10px;"')
gene #1289


select ps.id, ps.nom, ps.prenom, ar.dept_id, an.id adresse_norm_id, an.iris::float, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
where sp.specialite_id=10
and tds.date_source_id >= 2100 and  tds.date_source_id < 2200
and ar.dept_id=6
group by ps.id, c.id, ar.id, an.id



,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903.0,7.255881,43.709653
1,16130,BENSUSSAN,JEAN,6,92990,60882801.0,7.227639,43.676026
2,16131,LARA,JEANNINE,6,92963,60880202.0,7.282865,43.695573
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102.0,7.145627,43.657352
4,16148,CROQUET,PATRICE,6,1433,60690109.0,6.922279,43.651604
...,...,...,...,...,...,...,...,...
1281,155689,SAROUX,DELPHINE,6,2305,60882401.0,7.258125,43.695605
1282,155690,PERADOTTO,CYRIL,6,2866,60290121.0,6.988390,43.550361
1283,155692,DONADIO,MANON,6,77494,61490102.0,7.313438,43.743446
1284,155827,BOUCHET BADEL,MARIE,6,2328,60690102.0,6.914032,43.646318


In [3]:
gene["id"].nunique() # 1189 sql 1203

1203

In [4]:
gene["b"] = gene["id"].astype(str) + "_" + gene["lat"].astype(str) + "_" + gene["lon"].astype(str)
gene["b"].nunique() #1289 #sql 1274

1274

In [5]:
gene = gene.sort_values(by='id')
gene["un"] = gene.groupby("id")["b"].transform("nunique")
gene["un"].value_counts(normalize=True) # 85 12 1

un
1    0.891913
2    0.095645
3    0.009331
4    0.003110
Name: proportion, dtype: float64

In [6]:
gene['weight'] = 1 / gene['un']
gene.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,b,un,weight
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903.0,7.255881,43.709653,16126_43.709653_7.255881,1,1.0
1,16130,BENSUSSAN,JEAN,6,92990,60882801.0,7.227639,43.676026,16130_43.676026_7.227639,1,1.0
2,16131,LARA,JEANNINE,6,92963,60880202.0,7.282865,43.695573,16131_43.695573_7.282865,1,1.0
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102.0,7.145627,43.657352,16132_43.657352_7.145627,1,1.0
4,16148,CROQUET,PATRICE,6,1433,60690109.0,6.922279,43.651604,16148_43.651604_6.922279,1,1.0
5,16151,DESGATS,MARC,6,1436,60300107.0,7.005844,43.579045,16151_43.579045_7.005844,1,1.0
6,16153,DUBOIS,MICHEL,6,1438,61360000.0,7.449651,43.877046,16153_43.877046_7.449651,1,1.0
7,16159,GIDEL,PIERRE JEAN,6,1466,60882601.0,7.242220,43.689322,16159_43.689322_7.24222,1,1.0
8,16160,MASCHINO,JEAN MARC,6,1467,60290102.0,7.037066,43.546353,16160_43.546353_7.037066,1,1.0
9,16161,SARDOU,YVES,6,1468,60830205.0,7.492397,43.791639,16161_43.791639_7.492397,1,1.0


In [7]:
gene = gene.sort_values(by='iris')
gene["NB"] = gene.groupby("iris")["weight"].transform("sum")
gene

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,b,un,weight,NB
392,16965,CANOVA,JEAN PAUL,6,2236,404.0,7.428822,43.746168,16965_43.746168_7.428822,2,0.5,0.5
683,91280,BILLAUD,JEAN CHRISTOPHE,6,63066,60040102.0,7.070016,43.597006,91280_43.597006_7.070016,1,1.0,2.0
123,16425,DAEMERS,YVES,6,92989,60040102.0,7.072160,43.596934,16425_43.596934_7.07216,1,1.0,2.0
905,113924,DIAMANT,GUY,6,77467,60040103.0,7.137517,43.567521,113924_43.567521_7.137517,1,1.0,1.0
109,16404,MANDRILE,DIDIER,6,1651,60040104.0,7.092388,43.604533,16404_43.604533_7.092388,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1071,132333,AGOSTINI,PIERRE,6,271634,61610105.0,7.141885,43.640635,132333_43.640635_7.141885,2,0.5,5.0
1023,125010,LEBRUN CHAUVOT,EMILIE,6,271634,61610105.0,7.141885,43.640635,125010_43.640635_7.141885,2,0.5,5.0
301,16814,DI VINCENZO,DOMINIQUE,6,273951,61620000.0,7.612673,44.063665,16814_44.063665_7.612673,2,0.5,0.5
393,16965,CANOVA,JEAN PAUL,6,2743,61630000.0,7.587375,44.059283,16965_44.059283_7.587375,2,0.5,1.5


In [8]:
gene = gene.drop_duplicates(subset=['iris', 'NB'])
gene.to_csv("nbgpiris2022.csv", index=False)
gene

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,b,un,weight,NB
392,16965,CANOVA,JEAN PAUL,6,2236,404.0,7.428822,43.746168,16965_43.746168_7.428822,2,0.5,0.5
683,91280,BILLAUD,JEAN CHRISTOPHE,6,63066,60040102.0,7.070016,43.597006,91280_43.597006_7.070016,1,1.0,2.0
905,113924,DIAMANT,GUY,6,77467,60040103.0,7.137517,43.567521,113924_43.567521_7.137517,1,1.0,1.0
109,16404,MANDRILE,DIDIER,6,1651,60040104.0,7.092388,43.604533,16404_43.604533_7.092388,1,1.0,1.0
388,16957,NICASTRO,MAGALI,6,93020,60040105.0,7.117538,43.588105,16957_43.588105_7.117538,1,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
608,17388,ROUSSEAU,FRANCOISE,6,2635,61610103.0,7.123625,43.658047,17388_43.658047_7.123625,1,1.0,1.0
1022,125010,LEBRUN CHAUVOT,EMILIE,6,81986,61610104.0,7.141591,43.641591,125010_43.641591_7.141591,2,0.5,3.0
743,99028,BENASSAYAG,JEAN JACQUES,6,68080,61610105.0,7.140329,43.639824,99028_43.639824_7.140329,1,1.0,5.0
301,16814,DI VINCENZO,DOMINIQUE,6,273951,61620000.0,7.612673,44.063665,16814_44.063665_7.612673,2,0.5,0.5


In [9]:
min=30
dept=6
sql = f"""
(select iris.id "IRIS1", iris.id "IRIS2", 0 "KM", 0 "TIME" from iris.iris
where id / 10000000 = {dept}

union

select iris_id_from as "IRIS1", iris_id_to "IRIS2", route_km "KM", route_min "TIME" from iris.iris_matrix
where (iris_id_from/10000000={dept} or iris_id_to/10000000={dept})
and route_min <= {min}) order by "IRIS1", "IRIS2"
"""

# od06sql = pd.read_sql(sql, config.connection_string)
# od06sql.to_csv("od_06_sql.csv")
# od = od06sql
# od06sql

# od = pd.read_stata("Etude 06/ODFinale.dta")
# od = od[od["TIME"] <= 30]
# od #80712
# od

od = pd.read_csv("od_06_sql.csv")
od.style.set_table_attributes('style="font-size: 10px;height:10px;"')
od


,IRIS1,IRIS2,KM,TIME
0,40080000,60530000,17,22
1,40080000,60990000,21,24
2,40390000,61540000,24,30
3,40420000,60530000,11,19
4,40420000,60710000,22,30
...,...,...,...,...
117764,831380000,60850103,34,29
117765,831380000,60950101,18,19
117766,831380000,60950102,19,22
117767,831380000,60950103,18,20


In [10]:
pd.options.mode.copy_on_write = True
od["iris"] = od["IRIS2"].astype("int64")
od["iris"].nunique() # 500

566

In [11]:
from sqlalchemy import text
sql = f"""
select pi.iris, c.code "COM", i.type "TYP_IRIS", pi.p20_pop0002, pi.p20_pop0305, pi.p20_pop0610, pi.p20_pop1117, pi.p20_pop1824, pi.p20_pop2539, pi.p20_pop4559, pi.p20_pop6074, pi.p20_pop75p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '{dept:02}%'"""
print(sql)
pop_iris = pd.read_sql(text(sql), config.connection_string)
# pop_iris = pd.read_stata("Etude 06/popIRIS.dta")
pop_iris


select pi.iris, c.code "COM", i.type "TYP_IRIS", pi.p20_pop0002, pi.p20_pop0305, pi.p20_pop0610, pi.p20_pop1117, pi.p20_pop1824, pi.p20_pop2539, pi.p20_pop4559, pi.p20_pop6074, pi.p20_pop75p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '06%'


,iris,COM,TYP_IRIS,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p
0,060120105,06012,A,87.418550,60.438694,193.693660,273.95730,223.035700,676.78790,811.84125,457.24982,315.660160
1,060500000,06050,C,14.457494,14.409695,41.336414,48.39683,17.661858,72.61527,136.39720,95.22634,43.630077
2,060720000,06072,C,0.000000,7.000000,5.000000,5.00000,4.000000,22.00000,38.00000,35.00000,26.000000
3,061050101,06105,A,90.654580,136.365310,244.816740,402.70312,211.586750,430.94614,945.29596,679.35130,356.906370
4,060150000,06015,C,36.382305,39.335630,53.229595,83.62182,64.078064,198.46510,316.26556,265.24164,108.909730
...,...,...,...,...,...,...,...,...,...,...,...,...
495,060883403,06088,A,71.638420,55.635365,104.755210,138.66922,184.708200,294.65110,278.43094,337.78094,311.285340
496,060883501,06088,A,63.991177,86.144790,123.785240,215.96701,182.714100,349.32108,617.75810,466.61243,237.725800
497,060883601,06088,A,100.090430,73.343470,134.026950,215.99605,173.736830,455.11038,587.17010,585.84150,294.802900
498,060883602,06088,A,64.699010,79.082280,140.378510,183.99406,110.133000,398.01132,453.39966,453.67694,179.870680


In [12]:
pop_iris["iris"] = pd.to_numeric(pop_iris["iris"], errors="coerce")
od = od.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
od.head(10)

,IRIS1,IRIS2,KM,TIME,iris,COM,TYP_IRIS,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p
0,40080000,60530000,17,22,60530000,06053,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,23.000000,40.000000,26.000000
1,40080000,60990000,21,24,60990000,06099,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,365.603700,358.148500,209.747850
2,40390000,61540000,24,30,61540000,06154,C,14.443875,11.072821,23.354239,41.093530,36.612030,60.120470,105.712440,118.533870,45.796837
3,40420000,60530000,11,19,60530000,06053,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,23.000000,40.000000,26.000000
4,40420000,60710000,22,30,60710000,06071,C,13.000000,11.000000,30.000000,25.000000,21.000000,92.000000,118.000000,153.000000,90.000000
5,40420000,60990000,17,24,60990000,06099,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,365.603700,358.148500,209.747850
6,40420000,61240000,15,30,61240000,06124,C,0.941176,0.000000,1.882353,0.941176,0.941176,2.823529,11.294118,16.941177,10.352942
7,40430000,60020000,15,20,60020000,06002,C,0.962963,0.000000,0.962963,0.000000,4.814815,2.888889,15.407408,15.407408,9.629630
8,40430000,60240000,8,11,60240000,06024,C,4.785714,5.982143,4.785714,17.946428,3.589286,21.535715,34.696430,52.642857,41.875000
9,40430000,60450000,18,23,60450000,06045,C,0.000000,0.000000,2.730769,6.371795,1.820513,2.730769,20.025640,21.846153,10.923077


In [13]:
od["WexpGP"] = np.exp(-0.12 * od["TIME"])
od.to_csv("popiris2022.csv", index=False)
od.head(10)



,IRIS1,IRIS2,KM,TIME,iris,COM,TYP_IRIS,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p,WexpGP
0,40080000,60530000,17,22,60530000,06053,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,23.000000,40.000000,26.000000,0.071361
1,40080000,60990000,21,24,60990000,06099,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,365.603700,358.148500,209.747850,0.056135
2,40390000,61540000,24,30,61540000,06154,C,14.443875,11.072821,23.354239,41.093530,36.612030,60.120470,105.712440,118.533870,45.796837,0.027324
3,40420000,60530000,11,19,60530000,06053,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,23.000000,40.000000,26.000000,0.102284
4,40420000,60710000,22,30,60710000,06071,C,13.000000,11.000000,30.000000,25.000000,21.000000,92.000000,118.000000,153.000000,90.000000,0.027324
5,40420000,60990000,17,24,60990000,06099,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,365.603700,358.148500,209.747850,0.056135
6,40420000,61240000,15,30,61240000,06124,C,0.941176,0.000000,1.882353,0.941176,0.941176,2.823529,11.294118,16.941177,10.352942,0.027324
7,40430000,60020000,15,20,60020000,06002,C,0.962963,0.000000,0.962963,0.000000,4.814815,2.888889,15.407408,15.407408,9.629630,0.090718
8,40430000,60240000,8,11,60240000,06024,C,4.785714,5.982143,4.785714,17.946428,3.589286,21.535715,34.696430,52.642857,41.875000,0.267135
9,40430000,60450000,18,23,60450000,06045,C,0.000000,0.000000,2.730769,6.371795,1.820513,2.730769,20.025640,21.846153,10.923077,0.063292


In [14]:
od = od.sort_values(by='IRIS2')
# P19_POP0002 P19_POP0305 P19_POP0610 P19_POP1117 P19_POP1824 P19_POP2539 P19_POP4559 P19_POP6074 P19_POP75P
pop_cols = ["p20_pop0002", "p20_pop0305", "p20_pop0610", "p20_pop1117", "p20_pop1824", "p20_pop2539", "p20_pop4559", "p20_pop6074", "p20_pop75p"]
weights = [0.759201627]*4 + [0.784999993, 0.915, 1.05, 1.4, 1.3]
for col in pop_cols:
    od[col] = pd.to_numeric(od[col], errors="coerce")
od["POPGP"] = sum(w * od[c] for w, c in zip(weights, pop_cols))
od.to_csv("popiris2022.csv", index=False)
od.head(10)

,IRIS1,IRIS2,KM,TIME,iris,COM,TYP_IRIS,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p,WexpGP,POPGP
97682,60990000,40080000,21,24,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.056135,NaN
36626,60530000,40080000,17,22,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071361,NaN
108163,61540000,40390000,24,30,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027324,NaN
97683,60990000,40420000,17,24,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.056135,NaN
36627,60530000,40420000,11,19,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.102284,NaN
104581,61240000,40420000,15,30,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027324,NaN
41646,60710000,40420000,22,30,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027324,NaN
49069,60870000,40430000,18,22,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071361,NaN
97684,60990000,40430000,18,27,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039164,NaN
105210,61310000,40430000,22,28,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.034735,NaN


In [15]:
# KM iris COM TYP_IRIS MODIF_IRIS LAB_IRIS P19_POP P19_POP0002 P19_POP0305 P19_POP0610 P19_POP1117 P19_POP1824 P19_POP2539 P19_POP4054 P19_POP5564 P19_POP6579 P19_POP80P P19_POP0014 P19_POP1529 P19_POP3044 P19_POP4559 P19_POP6074 P19_POP75P P19_POP0019 P19_POP2064 P19_POP65P P19_POPH P19_H0014 P19_H1529 P19_H3044 P19_H4559 P19_H6074 P19_H75P P19_H0019 P19_H2064 P19_H65P P19_POPF P19_F0014 P19_F1529 P19_F3044 P19_F4559 P19_F6074 P19_F75P P19_F0019 P19_F2064 P19_F65P _merge WexpGP POPGP
od = od.drop(["KM","iris","COM"], axis=1)
for col in od.columns:
    if col.startswith("P20"):
        od = od.drop(col, axis=1)
od2 = od.drop(["WexpGP","POPGP", "TYP_IRIS"], axis=1)
od2.to_csv("ODgeneralistes06.csv", index=False)
od2


,IRIS1,IRIS2,TIME,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p
97682,60990000,40080000,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36626,60530000,40080000,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108163,61540000,40390000,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97683,60990000,40420000,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36627,60530000,40420000,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
39124,60690104,831380000,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30750,60300112,831380000,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38924,60690103,831380000,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31095,60300113,831380000,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
od["iris"] = od["IRIS1"]
od = od.sort_values(by='IRIS1')
nbgp = gene
od = od.merge(nbgp, on="iris", how="left", suffixes=('', ''))
od["NB"].unique()

array([        nan,  2.        ,  1.        ,  8.66666667,  4.        ,
        1.5       , 13.        ,  3.        ,  0.5       , 23.        ,
       11.5       ,  5.        ,  6.        , 10.        ,  7.        ,
        4.5       , 11.        , 14.        ,  7.5       , 17.        ,
        3.5       ,  7.16666667,  2.5       ,  6.5       , 10.5       ,
        3.16666667,  1.25      ,  8.        , 13.5       , 12.5       ,
       25.66666667,  8.5       ,  9.        , 11.66666667,  7.25      ,
        5.33333333,  5.58333333,  9.5       ])

In [17]:
od["NB"] = od["NB"].fillna(0)
od = od.sort_values(by=['IRIS1', "IRIS2"])
# od.to_csv("IRIS2022.csv", index=False)

In [18]:
od = od.sort_values(by='IRIS1')
od["wpop"] = od["WexpGP"] * od["POPGP"]
od["swpop"] = od.groupby("IRIS1")["wpop"].transform("sum")
od["R"] = od["NB"] / (od["swpop"] / 100000)
od.head(10)

,IRIS1,IRIS2,TIME,TYP_IRIS,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p,WexpGP,POPGP,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,b,un,weight,NB,wpop,swpop,R
1,40080000,60530000,22,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,23.000000,40.000000,26.000000,0.071361,151.910839,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.840550,112.133197,0.0
0,40080000,60990000,24,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,365.603700,358.148500,209.747850,0.056135,1804.454871,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,101.292646,112.133197,0.0
2,40390000,61540000,30,C,14.443875,11.072821,23.354239,41.093530,36.612030,60.120470,105.712440,118.533870,45.796837,0.027324,488.533210,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.348546,13.348546,0.0
3,40420000,60530000,19,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,23.000000,40.000000,26.000000,0.102284,151.910839,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.538080,135.163789,0.0
5,40420000,60710000,30,C,13.000000,11.000000,30.000000,25.000000,21.000000,92.000000,118.000000,153.000000,90.000000,0.027324,615.741928,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16.824362,135.163789,0.0
4,40420000,60990000,24,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,365.603700,358.148500,209.747850,0.056135,1804.454871,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,101.292646,135.163789,0.0
6,40420000,61240000,30,C,0.941176,0.000000,1.882353,0.941176,0.941176,2.823529,11.294118,16.941177,10.352942,0.027324,55.215820,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.508702,135.163789,0.0
12,40430000,61310000,28,C,0.979592,0.000000,0.000000,2.938776,2.938776,4.897959,13.714286,16.653060,5.877551,0.034735,55.118503,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.914555,157.125891,0.0
14,40430000,61160000,26,C,5.785167,2.835322,5.693009,12.499444,5.853335,18.495186,56.672413,61.155640,34.253048,0.044157,231.527285,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.223589,157.125891,0.0
7,40430000,60870000,22,C,0.000000,0.000000,1.000000,1.000000,4.000000,3.000000,10.000000,11.000000,9.000000,0.071361,45.003403,40430000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.211500,157.125891,0.0


In [19]:
rgp = od[od["IRIS1"] == od["IRIS2"]]
rgp = rgp[["IRIS1", "TYP_IRIS", "POPGP", "NB", "R"]].copy()
rgp.head(10)

,IRIS1,TYP_IRIS,POPGP,NB,R
76,60010000,C,92.810629,0.000000,0.000000
84,60020000,C,58.151798,0.000000,0.000000
107,60030000,C,625.817363,0.000000,0.000000
274,60040101,C,1559.693523,0.000000,0.000000
492,60040102,A,2368.339365,2.000000,1.445919
848,60040103,A,858.108351,1.000000,1.283253
1396,60040104,A,2059.816516,1.000000,0.615765
1549,60040105,A,3898.006226,2.000000,1.486136
1878,60040106,A,2729.254386,8.666667,6.675589
2249,60040107,A,3125.706245,4.000000,2.890353


In [20]:
rgp["IRIS2"] = rgp["IRIS1"]
rgp.to_csv("RGP2022.csv", index=False)
od = od.merge(rgp, on="IRIS2", suffixes=("", "_dest"))
od.head(10)

,IRIS1,IRIS2,TIME,TYP_IRIS,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p,WexpGP,POPGP,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,b,un,weight,NB,wpop,swpop,R,IRIS1_dest,TYP_IRIS_dest,POPGP_dest,NB_dest,R_dest
0,40080000,60530000,22,C,3.0,3.0,3.0,15.0,3.0,19.0,23.0,40.0,26.0,0.071361,151.910839,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.840550,112.133197,0.000000,60530000,C,151.910839,0.0,0.0
1,40420000,60530000,19,C,3.0,3.0,3.0,15.0,3.0,19.0,23.0,40.0,26.0,0.102284,151.910839,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.538080,135.163789,0.000000,60530000,C,151.910839,0.0,0.0
2,40760000,60530000,19,C,3.0,3.0,3.0,15.0,3.0,19.0,23.0,40.0,26.0,0.102284,151.910839,40760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.538080,723.053377,0.000000,60530000,C,151.910839,0.0,0.0
3,40900000,60530000,28,C,3.0,3.0,3.0,15.0,3.0,19.0,23.0,40.0,26.0,0.034735,151.910839,40900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.276662,54.581086,0.000000,60530000,C,151.910839,0.0,0.0
4,41740000,60530000,20,C,3.0,3.0,3.0,15.0,3.0,19.0,23.0,40.0,26.0,0.090718,151.910839,41740000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.781040,142.549231,0.000000,60530000,C,151.910839,0.0,0.0
5,42020000,60530000,9,C,3.0,3.0,3.0,15.0,3.0,19.0,23.0,40.0,26.0,0.339596,151.910839,42020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,51.588241,315.111601,0.000000,60530000,C,151.910839,0.0,0.0
6,60530000,60530000,0,C,3.0,3.0,3.0,15.0,3.0,19.0,23.0,40.0,26.0,1.000000,151.910839,60530000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,151.910839,359.033874,0.000000,60530000,C,151.910839,0.0,0.0
7,60710000,60530000,18,C,3.0,3.0,3.0,15.0,3.0,19.0,23.0,40.0,26.0,0.115325,151.910839,60710000,16333.0,BOFFY,FRANCOIS,6.0,1937.0,6.852913,44.091130,16333_44.09113_6.852913,1.0,1.0,1.0,17.519136,1070.485700,93.415540,60530000,C,151.910839,0.0,0.0
8,60940000,60530000,25,C,3.0,3.0,3.0,15.0,3.0,19.0,23.0,40.0,26.0,0.049787,151.910839,60940000,91295.0,GIRAUD,ROLAND,6.0,63080.0,6.932793,44.094193,91295_44.094193_6.932793,1.0,1.0,1.0,7.563195,1227.881541,81.441081,60530000,C,151.910839,0.0,0.0
9,60990000,60530000,26,C,3.0,3.0,3.0,15.0,3.0,19.0,23.0,40.0,26.0,0.044157,151.910839,60990000,102698.0,DIETZ,JEAN PAUL,6.0,98683.0,6.898646,43.953725,102698_43.953725_6.898646,1.0,1.0,1.0,6.707953,2139.644821,46.736729,60530000,C,151.910839,0.0,0.0


In [21]:
od = od.sort_values(by=['IRIS1', "IRIS2"]).copy()
od["Ap"] = od["WexpGP"] * od["R_dest"]
od["APL"] = od.groupby("IRIS1")["Ap"].transform("sum")
od

,IRIS1,IRIS2,TIME,TYP_IRIS,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p,WexpGP,POPGP,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,b,un,weight,NB,wpop,swpop,R,IRIS1_dest,TYP_IRIS_dest,POPGP_dest,NB_dest,R_dest,Ap,APL
0,40080000,60530000,22,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,23.00000,40.00000,26.000000,0.071361,151.910839,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.840550,112.133197,0.0,60530000,C,151.910839,0.0,0.000000,0.000000,2.623555
12,40080000,60990000,24,C,45.961033,60.562077,118.186980,174.26068,100.920586,288.92783,365.60370,358.14850,209.747850,0.056135,1804.454871,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,101.292646,112.133197,0.0,60990000,C,1804.454871,1.0,46.736729,2.623555,2.623555
33,40390000,61540000,30,C,14.443875,11.072821,23.354239,41.09353,36.612030,60.12047,105.71244,118.53387,45.796837,0.027324,488.533210,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.348546,13.348546,0.0,61540000,C,488.533210,1.0,110.010389,3.005893,3.005893
1,40420000,60530000,19,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,23.00000,40.00000,26.000000,0.102284,151.910839,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.538080,135.163789,0.0,60530000,C,151.910839,0.0,0.000000,0.000000,5.176015
51,40420000,60710000,30,C,13.000000,11.000000,30.000000,25.00000,21.000000,92.00000,118.00000,153.00000,90.000000,0.027324,615.741928,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16.824362,135.163789,0.0,60710000,C,615.741928,1.0,93.415540,2.552460,5.176015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87713,831380000,60850103,29,A,71.068900,71.788710,146.845930,194.39722,148.091400,351.81476,559.87990,510.06640,392.299470,0.030807,2617.648504,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,80.642973,5969.479466,0.0,60850103,A,2617.648504,0.0,0.000000,0.000000,7.505886
49230,831380000,60950101,19,A,78.000000,88.000000,211.000000,256.00000,155.000000,476.00000,682.00000,642.00000,411.000000,0.102284,3186.989629,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,325.978706,5969.479466,0.0,60950101,A,3186.989629,0.0,0.000000,0.000000,7.505886
55000,831380000,60950102,22,A,68.000000,78.000000,156.000000,226.00000,145.000000,400.00000,530.00000,485.00000,293.000000,0.071361,2497.083458,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,178.195046,5969.479466,0.0,60950102,A,2497.083458,6.0,22.161238,1.581454,7.505886
54896,831380000,60950103,20,A,47.000000,69.000000,126.000000,225.00000,103.000000,259.00000,567.00000,576.00000,327.000000,0.090718,2499.237159,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,226.725680,5969.479466,0.0,60950103,A,2499.237159,0.0,0.000000,0.000000,7.505886


In [22]:
apl_final = od[od["IRIS1"] == od["IRIS2"]]
apl_final.head(10)

,IRIS1,IRIS2,TIME,TYP_IRIS,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4559,p20_pop6074,p20_pop75p,WexpGP,POPGP,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,b,un,weight,NB,wpop,swpop,R,IRIS1_dest,TYP_IRIS_dest,POPGP_dest,NB_dest,R_dest,Ap,APL
385,60010000,60010000,0,C,4.000000,4.000000,9.000000,1.000000,2.000000,15.000000,19.000000,23.000000,9.00000,1.0,92.810629,60010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,92.810629,279.581724,0.000000,60010000,C,92.810629,0.000000,0.000000,0.000000,34.478165
157,60020000,60020000,0,C,0.962963,0.000000,0.962963,0.000000,4.814815,2.888889,15.407408,15.407408,9.62963,1.0,58.151798,60020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,58.151798,237.808633,0.000000,60020000,C,58.151798,0.000000,0.000000,0.000000,0.000000
242,60030000,60030000,0,C,9.752499,21.342184,46.559483,61.267956,36.527490,93.859300,152.356190,144.725160,33.23300,1.0,625.817363,60030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,625.817363,1200.693332,0.000000,60030000,C,625.817363,0.000000,0.000000,0.000000,18.184595
74748,60040101,60040101,0,C,43.067524,29.473127,53.949734,52.204407,157.987120,374.808720,398.335300,311.410300,79.10138,1.0,1559.693523,60040101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1559.693523,173102.022577,0.000000,60040101,C,1559.693523,0.000000,0.000000,0.000000,153.504937
74398,60040102,60040102,0,A,124.795586,79.509380,182.162320,236.056400,135.785890,461.605650,523.300100,404.160200,193.43684,1.0,2368.339365,60040102,91280.0,BILLAUD,JEAN CHRISTOPHE,6.0,63066.0,7.070016,43.597006,91280_43.597006_7.070016,1.0,1.000000,2.000000,2368.339365,138320.367797,1.445919,60040102,A,2368.339365,2.000000,1.445919,1.445919,122.817435
74130,60040103,60040103,0,A,23.252556,10.911196,29.277357,27.889765,53.339554,90.759460,194.654140,153.112560,188.54532,1.0,858.108351,60040103,113924.0,DIAMANT,GUY,6.0,77467.0,7.137517,43.567521,113924_43.567521_7.137517,1.0,1.000000,1.000000,858.108351,77926.962793,1.283253,60040103,A,858.108351,1.000000,1.283253,1.283253,74.591688
83097,60040104,60040104,0,A,50.537685,64.513460,99.013300,179.561550,72.332810,290.570470,454.758100,429.753140,276.28568,1.0,2059.816516,60040104,16404.0,MANDRILE,DIDIER,6.0,1651.0,7.092388,43.604533,16404_43.604533_7.092388,1.0,1.000000,1.000000,2059.816516,162399.567258,0.615765,60040104,A,2059.816516,1.000000,0.615765,0.615765,144.592217
73476,60040105,60040105,0,A,121.733350,125.041620,158.855360,180.071910,239.746570,751.015870,680.204960,814.698800,556.28156,1.0,3898.006226,60040105,16957.0,NICASTRO,MAGALI,6.0,93020.0,7.117538,43.588105,16957_43.588105_7.117538,1.0,1.000000,2.000000,3898.006226,134577.209269,1.486136,60040105,A,3898.006226,2.000000,1.486136,1.486136,119.254520
73130,60040106,60040106,0,A,85.542570,70.825040,88.412636,137.204270,96.797050,420.406620,533.387400,576.493800,470.34253,1.0,2729.254386,60040106,17532.0,PICOT,CHRISTOPHE,6.0,2734.0,7.112369,43.606074,17532_43.606074_7.112369,3.0,0.333333,8.666667,2729.254386,129826.249475,6.675589,60040106,A,2729.254386,8.666667,6.675589,6.675589,118.846147
72782,60040107,60040107,0,A,144.588610,82.239660,149.386700,173.899250,189.906750,781.107850,579.516900,600.839700,303.53833,1.0,3125.706245,60040107,17295.0,BOUTINOT,LAURENT,6.0,273638.0,7.121580,43.600582,17295_43.600582_7.12158,2.0,0.500000,4.000000,3125.706245,138391.414923,2.890353,60040107,A,3125.706245,4.000000,2.890353,2.890353,124.730495


In [23]:
apl_final["APL"].describe() #mean=98 median=108 du 2205 au 2305

count    500.000000
mean     103.760833
std       55.012623
min        0.000000
25%       66.019785
50%      110.003455
75%      146.325274
max      509.340659
Name: APL, dtype: float64

In [24]:
apl_final = apl_final[["IRIS1", "TYP_IRIS", "NB", "APL"]]
apl_final.head(10)

,IRIS1,TYP_IRIS,NB,APL
385,60010000,C,0.000000,34.478165
157,60020000,C,0.000000,0.000000
242,60030000,C,0.000000,18.184595
74748,60040101,C,0.000000,153.504937
74398,60040102,A,2.000000,122.817435
74130,60040103,A,1.000000,74.591688
83097,60040104,A,1.000000,144.592217
73476,60040105,A,2.000000,119.254520
73130,60040106,A,8.666667,118.846147
72782,60040107,A,4.000000,124.730495


In [25]:
sql = f"""
select i.id "IRIS1", i.code "IRIS", i.nom "LIB_IRIS", c.dept_id "DEP", c.code "DEPCOM", c.nom "LIBCOM", 20{year} "YEAR", {specialite} "SPECIALITE" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '{dept:02d}%'"""
print(sql)
nom_iris = pd.read_sql(text(sql), config.connection_string)
# nom_iris = pd.read_stata("Etude 06/NomIRIS.dta")
nom_iris



select i.id "IRIS1", i.code "IRIS", i.nom "LIB_IRIS", c.dept_id "DEP", c.code "DEPCOM", c.nom "LIBCOM", 2021 "YEAR", 10 "SPECIALITE" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '06%'


,IRIS1,IRIS,LIB_IRIS,DEP,DEPCOM,LIBCOM,YEAR,SPECIALITE
0,60120105,060120105,Centre,6,06012,Beausoleil,2021,10
1,60500000,060500000,Coursegoules,6,06050,Coursegoules,2021,10
2,60720000,060720000,Ilonse,6,06072,Ilonse,2021,10
3,61050101,061050101,Nord,6,06105,Roquefort-les-Pins,2021,10
4,60150000,060150000,Berre-les-Alpes,6,06015,Berre-les-Alpes,2021,10
...,...,...,...,...,...,...,...,...
495,60883403,060883403,Madeleine Supérieure,6,06088,Nice,2021,10
496,60883501,060883501,Crémat-Bellet,6,06088,Nice,2021,10
497,60883601,060883601,Saint-Pierre de Féric,6,06088,Nice,2021,10
498,60883602,060883602,Costière,6,06088,Nice,2021,10


In [29]:
apl_final = apl_final.merge(nom_iris, on="IRIS1", how="left", suffixes=("", "_dest"))
apl_final = apl_final[["IRIS", "TYP_IRIS", "NB", "APL", "LIB_IRIS", "DEP", "DEPCOM", "LIBCOM", "YEAR", "SPECIALITE"]]
apl_final

,IRIS,TYP_IRIS,NB,APL,LIB_IRIS,DEP,DEPCOM,LIBCOM,YEAR,SPECIALITE
0,060010000,C,0.0,34.478165,Aiglun,6,06001,Aiglun,2021,10
1,060020000,C,0.0,0.000000,Amirat,6,06002,Amirat,2021,10
2,060030000,C,0.0,18.184595,Andon,6,06003,Andon,2021,10
3,060040101,C,0.0,153.504937,Les Trois Moulins,6,06004,Antibes,2021,10
4,060040102,A,2.0,122.817435,Les Semboules,6,06004,Antibes,2021,10
...,...,...,...,...,...,...,...,...,...,...
495,061610105,A,5.0,161.018274,Marina,6,06161,Villeneuve-Loubet,2021,10
496,061610106,A,0.0,116.616400,Vaugrenier,6,06161,Villeneuve-Loubet,2021,10
497,061610107,A,0.0,128.445387,Plan,6,06161,Villeneuve-Loubet,2021,10
498,061620000,C,0.5,58.950099,La Brigue,6,06162,La Brigue,2021,10


In [27]:
apl_final.to_csv("APL06_2022_Final.csv", index=False)

In [28]:
# TODO trouver les ratio DAMIR
# TODO remplacer ps par personne_activite
# TODO Ajouter datesource_id à pa du genre pa_cabinet_datesource + adresse_raw + adresse_norm
# TODO si adresse vide faire jointure ps, ajouter une colonne is_adresse_empty=True
# Sinon pour trouver chaque spé il faut filtrer sur codeprofession==10 et ensuite travailler avec la variable libellésavoirfaire (tu as plusieurs regroupement à faire pour les gyéncos obstétriciens par ex ou pour les généralistes